In [0]:
from pyspark.sql import SparkSession, functions as f

In [0]:
dbutils.fs.mounts()

In [0]:
df_hosa=spark.read.parquet("/mnt/rcmabhi/bronze/hosa/departments")

In [0]:
display(df_hosa)

In [0]:
df_hosb=spark.read.parquet("/mnt/rcmabhi/bronze/hosb/departments")

In [0]:
df_merged = df_hosa.unionByName(df_hosb)

In [0]:
df_merged = df_merged.withColumn("SRC_Dept_id", f.col("deptid")) \
                     .withColumn("Dept_id", f.concat(f.col("deptid"),f.lit('-'), f.col("datasource"))) \
                     .drop("deptid")


In [0]:
display(df_merged)

In [0]:
%sql
create schema if not exists silver
location "/mnt/rcmabhi/silver"

In [0]:
%sql
show schemas

In [0]:
%sql
CREATE TABLE silver.departments (
  Dept_Id STRING,
  SRC_Dept_Id STRING,
  Name STRING,
  datasource STRING,
  is_quarantined BOOLEAN
)
USING DELTA

In [0]:
%sql
describe extended silver.departments

In [0]:
df_final = df_merged.withColumn(
    "is_quarantined",
    f.when(f.col("SRC_Dept_id").isNull() | f.col("Name").isNull(), f.lit(True)).otherwise(f.lit(False))
).select("Dept_id", "SRC_Dept_id", "Name", "datasource", "is_quarantined")


In [0]:
df_final.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("silver.departments")

In [0]:
display(df_final)